In [1]:
import os
import requests
import base64
import json
import csv
import shutil
from typing import List
from natsort import os_sorted

In [2]:
# Configuration
GPT4V_ENDPOINT = "YOUR_API_ENDPOINT"
GPT4V_KEY = "YOUR_API_KEY"
headers = {
    "Content-Type": "application/json",
    "api-key": GPT4V_KEY,
}

In [3]:
def extract_evaluate_content(image_path:str, extract_list:List[str]):
    encoded_image = base64.b64encode(open(image_path, 'rb').read()).decode('ascii')

    system_prompt = """
    You are an AI assistant that helps extract text from images of spreadsheets.
    You will be given the list of columns to extract text from.
    You should extract text from each column of the spreadsheet.
    Reply only with a JSON object.
    The JSON object should be an array of objects where each object represents a row in the spreadsheet.
    The keys of the JSON must be the column number.
    Within strings, you must replace all double-quotes with single quotes.

    For example:
    [
        {
            "1": "ESRS2.94GOV-18",
            "2": "GOV-1",
            "3": "21 (e)",
            "4": "N",
            "5": "%",
            "6": "-"
        }
    ]
    """
    user_prompt = """
    extract the following columns from the image.
    columns: {0}
    """.format(extract_list)

    payload = {
    "messages": [
        {
        "role": "system",
        "content": [
            {
            "type": "text",
            "text": system_prompt
            }
        ]
        },
        {
        "role": "user",
        "content": [
            {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{encoded_image}",
                "detail": "high"
            }
            },
            {
            "type": "text",
            "text": user_prompt
            }
        ]
        }
    ],
    "temperature": 0.0,
    "top_p": 0.0,
    "max_tokens": 4096
    }

    try:
        response = requests.post(GPT4V_ENDPOINT, headers=headers, json=payload)
        response.raise_for_status()
    except requests.RequestException as e:
        raise SystemExit(f"Failed to make the request. Error: {e}")
    response_json = response.json()
    print(f"Tokens: {response_json['usage']}")
    result=response_json['choices'][0]['message']['content']
    try:
        # if the result contains json then remove it
        result=result.replace("json","")
        result=result.replace("```","")
        result=result.replace("\n","")
        result=result.replace("\\","")
        result=json.loads(result)
    except Exception as e:
        print(f"Error extract_evaluate_content: {e}")
        result=result
    return result

In [4]:
def replace_keys_in_list(dicts, key_mapping):
    for d in dicts:
        for old_key, new_key in key_mapping.items():
            if old_key in d:
                d[new_key] = d.pop(old_key)
    return dicts

In [6]:
key_mapping = {
    "1": "Unique Element ID 17/05",
    "2": "Description"
}

# key_mapping = {
#     "1": "Unique Element ID 17/05",
#     "2": "Data Element Unit",
#     "3": "Defined Terms",
#     "4": "Interpretation/ external references"
# }

# key_mapping = {
#     "1": "Unique Element ID 17/05",
#     "2": "Datapoint",
#     "3": "Mandatory calc methodology",
#     "4": "Calculation methodology",
#     "5": "Unit (Quantitative only)",
#     "6": "Datapoint Description",
#     "7": "Data Element Unit"
# }

In [ ]:
all_data = []

os.makedirs('merged', exist_ok=True)

files = os.listdir()

files = os_sorted(files)

for filename_ext in files:
    try:
        if filename_ext.endswith(('.png')):
            filename = filename_ext.split(".")[0]
            print(filename)

            input_string = extract_evaluate_content(filename_ext, ["Unique Element ID 17/05","Description"])
            # input_string = extract_evaluate_content(filename_ext, ["Unique Element ID 17/05","Data Element Unit","Defined Terms", "Interpretation/ external references"])
            # input_string = extract_evaluate_content(filename_ext, ["Unique Element ID 17/05","Datapoint","Mandatory calc methodology", "Calculation methodology","Unit (Quantitative only)","Datapoint Description","Data Element Unit"])
            print(input_string)

            replace_json_data = replace_keys_in_list(input_string, key_mapping)

            all_data.append(replace_json_data)
            shutil.move(filename_ext, f'merged/{filename_ext}')

    except Exception as e:
        print(f"Error: {e}")
        break

keys = all_data[0][0].keys()

filename_csv = 'merged.csv'
with open(filename_csv, 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, fieldnames=keys)
    dict_writer.writeheader()
    for data in all_data:
        dict_writer.writerows(data)